In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
df = pd.read_csv(r'/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv(r'/kaggle/input/digit-recognizer/test.csv')
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split

We are going to split data with test size 20%

In [4]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('label', axis = 1), df['label'], test_size = 0.2)

In [5]:
# Let's print the shape of data after train test split
x_train.shape, x_test.shape

((33600, 784), (8400, 784))

* To pass to our model we need to change the shape of our dataset

In [6]:
x_train = np.array(x_train).reshape(x_train.shape[0], 28, 28, 1)
x_test = np.array(x_test).reshape(x_test.shape[0], 28, 28, 1)

test_array =  np.array(test).reshape(test.shape[0], 28, 28, 1)

In [7]:
x_train = x_train / 255.0

x_test = x_test / 255.0

test_array = test_array / 255.0

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
test_array = test_array.astype('float32')

In [8]:
#Convert class vector to binary class matrices, It's simply like a one hot encoding
y_train = keras.utils.to_categorical(y_train, 10)

y_test = keras.utils.to_categorical(y_test, 10)

In [9]:
model = keras.models.Sequential()

model.add(keras.layers.Convolution2D(20, (5,5), activation = 'relu', input_shape = (28, 28, 1)))
model.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))

model.add(keras.layers.Convolution2D(50,(5,5), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(200, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 20)        520       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 50)          25050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 200)               160200    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2

In [11]:
model.compile(
    loss = keras.losses.categorical_crossentropy,
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy'])

In [12]:
history = model.fit(x_train, y_train, 
                   epochs = 12 , 
                   verbose = True,
                    batch_size= 32,
                   validation_split= 0.95)

Epoch 1/12
53/53 [==============================] - 10s 55ms/step - loss: 1.6332 - accuracy: 0.4585 - val_loss: 0.4199 - val_accuracy: 0.8670
Epoch 2/12
53/53 [==============================] - 2s 46ms/step - loss: 0.3176 - accuracy: 0.8993 - val_loss: 0.2677 - val_accuracy: 0.9162
Epoch 3/12
53/53 [==============================] - 2s 39ms/step - loss: 0.1767 - accuracy: 0.9492 - val_loss: 0.2216 - val_accuracy: 0.9302
Epoch 4/12
53/53 [==============================] - 2s 40ms/step - loss: 0.1235 - accuracy: 0.9676 - val_loss: 0.1779 - val_accuracy: 0.9462
Epoch 5/12
53/53 [==============================] - 2s 40ms/step - loss: 0.0658 - accuracy: 0.9851 - val_loss: 0.1957 - val_accuracy: 0.9398
Epoch 6/12
53/53 [==============================] - 2s 39ms/step - loss: 0.0468 - accuracy: 0.9926 - val_loss: 0.1529 - val_accuracy: 0.9552
Epoch 7/12
53/53 [==============================] - 2s 43ms/step - loss: 0.0550 - accuracy: 0.9912 - val_loss: 0.1526 - val_accuracy: 0.9570
Epoch 8/12
5

In [13]:
model.evaluate(x_test, y_test)

263/263 [==============================] - 1s 2ms/step - loss: 0.1548 - accuracy: 0.9608


[0.1548088788986206, 0.9608333110809326]

In [14]:
pred = model.predict(test_array)

In [15]:
sub = pd.DataFrame()
sub['ImageId'] = np.arange(1 , len(test)+1, 1)
sub['Label'] = np.argmax(pred, axis = 1)
sub.to_csv('./Submission.csv', index = False)